In [ ]:
from flask import Flask
import boto3

app = Flask(__name__)

# Initialize EC2 resource in Singapore region
myec2 = boto3.resource("ec2", region_name="ap-southeast-1")

# Store launched instance IDs
launched_instance_ids = []

@app.route("/os")
def osLaunch():
    instances = myec2.create_instances(
        InstanceType="t2.micro",
        ImageId="ami-0a3ece531caa5d49d",
        MinCount=1,
        MaxCount=1
    )

    instance = instances[0]
    instance.wait_until_running()
    instance.load()  # refresh to get public IP

    launched_instance_ids.append(instance.id)

    return (
        f"✅ Instance Launched!\n"
        f"🆔 ID: {instance.id}\n"
        f"🌐 Public IP: {instance.public_ip_address}\n"
        f"🔁 State: {instance.state['Name']}\n"
        f"📍 Region: ap-southeast-1"
    )

@app.route("/dlt")
def osDelete():
    if not launched_instance_ids:
        return "❌ No instances launched to delete."

    output = "🗑 Terminating instances:\n\n"

    for instance_id in launched_instance_ids:
        instance = myec2.Instance(instance_id)
        instance.terminate()
        instance.wait_until_terminated()
        instance.load()
        output += (
            f"✅ Terminated ID: {instance.id}\n"
            f"🔁 Final State: {instance.state['Name']}\n\n"
        )

    launched_instance_ids.clear()
    return output



if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple("localhost", 5000, app, use_debugger=True)

 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Jul/2025 01:54:29] "GET /os HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2025 01:54:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Jul/2025 01:59:01] "GET /dlt HTTP/1.1" 200 -
